In [388]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from tensorflow.keras.optimizers import SGD
from keras.utils import to_categorical
from keras.models import Sequential
from time import time

In [389]:
# Import dataset
train = pd.read_csv('optdigits.tra', header=None)
test = pd.read_csv('optdigits.tes', header=None)

In [390]:
# Sort x and y
train_x = train.iloc[:, 0:64]
train_y = train[64]
test_x = test.iloc[:, 0:64]
test_y = test[64]

In [391]:
# Standardization
train_x = train_x.astype('float32') / 16.
test_x = test_x.astype('float32') / 16.

In [392]:
# Split training set into training set and validation set
new_train_x, val_x, new_train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=1)

In [393]:
# Changed dataset into numpy
new_train_x = new_train_x.to_numpy()
new_train_y = new_train_y.to_numpy()
val_x = val_x.to_numpy()
val_y = val_y.to_numpy()
test_x = test_x.to_numpy()
test_y = test_y.to_numpy()

In [394]:
# Change the shape of y data set
new_train_y = to_categorical(new_train_y)
test_y = to_categorical(test_y)
val_y = to_categorical(val_y)

In [395]:
# Creat a model
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(30, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [396]:
# Set SGD as the optimizer
opt = SGD(learning_rate=0.15, momentum=0.15, nesterov=False)

In [397]:
# Early Stopping
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                              min_delta=0, patience=0, 
                              verbose=0, mode='auto')

In [398]:
# Model Compiling
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [399]:
# Fit Model
start = time()
model.fit(new_train_x, new_train_y, epochs= 180, batch_size=15,
          callbacks = [early_stop], validation_data=(val_x, val_y))
print(time()-start)

Train on 3058 samples, validate on 765 samples
Epoch 1/180
3058/3058 [==============================] - 1s 302us/sample - loss: 0.7327 - accuracy: 0.8103 - val_loss: 0.2559 - val_accuracy: 0.9373
Epoch 2/180
3058/3058 [==============================] - 0s 163us/sample - loss: 0.1945 - accuracy: 0.9460 - val_loss: 0.1997 - val_accuracy: 0.9477
Epoch 3/180
3058/3058 [==============================] - 0s 150us/sample - loss: 0.1397 - accuracy: 0.9621 - val_loss: 0.1472 - val_accuracy: 0.9595
Epoch 4/180
3058/3058 [==============================] - 0s 137us/sample - loss: 0.1173 - accuracy: 0.9637 - val_loss: 0.1404 - val_accuracy: 0.9621
Epoch 5/180
3058/3058 [==============================] - 1s 193us/sample - loss: 0.1007 - accuracy: 0.9729 - val_loss: 0.1236 - val_accuracy: 0.9647
Epoch 6/180
3058/3058 [==============================] - 1s 232us/sample - loss: 0.0879 - accuracy: 0.9781 - val_loss: 0.1439 - val_accuracy: 0.9608
3.7273271083831787


In [400]:
# Distribution of training set  >>> counted with for loops
dist_train = [376,389,380,389,387,376,377,387,380,382]
# Distribution of testing set
dist_test = [178,182,177,183,181,182,181,179,174,180]

In [401]:
# Accuracay of overall classification accuracy and class accuracy
def accuracy_overall(pred, actual, num_classes):
    result = int()
# Overall classfication accuracy
    overall = np.where(pred == actual)[0]
    overall_acc = float(len(overall))/len(pred)
    result = overall_acc
    return result

def accuracy(pred, actual, num_classes):
# Class accuracy
    result = []
    classes = np.array([float(0)]*10)
    for i in range(0,len(pred)):
        id_classes = int(actual[i])
        if pred[i] == actual[i]:
            classes[id_classes] = classes[id_classes] + 1
    classes_acc = classes/num_classes
    result.append(classes_acc)
    return result

In [402]:
# confusion matrix
def confusion_matrix(pred, actual):
    matrix = np.array([int(0)]*10*10).reshape(10,10)
    for i in range(0,10):
        pred_i = pred[np.where(np.array(actual) == i)]
        conf = []
        for j in range(0,10):
            length = len(np.where(pred_i == j)[0])
            conf.append(length)
        matrix[i,] = conf
    return matrix

In [403]:
test_y = test[64]
test_y = test_y.to_numpy()

In [404]:
predict_train = model.predict(train_x)
predict_train_class = predict_train.argmax(axis=-1)
acc_train_overall = accuracy_overall(predict_train_class, train_y, dist_train)
acc_train_class = accuracy(predict_train_class, train_y, dist_train)
conf_train = confusion_matrix(predict_train_class, train_y)


predict_test = model.predict(test_x)
predict_test_class = predict_test.argmax(axis=-1)
acc_test_overall = accuracy_overall(predict_test_class, test_y, dist_test)
acc_test_class = accuracy(predict_test_class, test_y, dist_test)
conf_test = confusion_matrix(predict_test_class, test_y)

In [405]:
#output results
print("training set overall accuracy" + "\n")
print(acc_train_overall)
print("\n")
print("training set class accuracy" + "\n")
print(acc_train_class)
print("\n")
print("training set confusion matrix" + "\n")
print(conf_train)
print("\n")
print("testing set overall accuracy" + "\n")
print(acc_test_overall)
print("\n")
print("testing set class accuracy" + "\n")
print(acc_test_class)
print("\n")
print("testing set confusion matrix" + "\n")
print(conf_test)

training set overall accuracy

0.970180486528904


training set class accuracy

[array([0.99468085, 0.98971722, 0.98684211, 0.96143959, 0.91731266,
       0.98138298, 0.95225464, 0.99224806, 0.96842105, 0.95811518])]


training set confusion matrix

[[374   0   0   0   1   0   0   0   1   0]
 [  0 385   0   0   0   0   0   1   1   2]
 [  1   1 375   0   0   0   0   0   2   1]
 [  0   0   2 374   0   4   0   1   4   4]
 [  1  12   0   0 355   0   1   0   3  15]
 [  0   1   2   1   0 369   0   0   0   3]
 [  2   9   0   0   2   1 359   0   4   0]
 [  0   0   1   1   0   0   0 384   0   1]
 [  1  10   0   0   0   1   0   0 368   0]
 [  0   6   0   0   2   1   0   2   5 366]]


testing set overall accuracy

0.9326655537006121


testing set class accuracy

[array([0.98314607, 0.98351648, 0.95480226, 0.89071038, 0.87845304,
       0.97802198, 0.92265193, 0.88826816, 0.8908046 , 0.95555556])]


testing set confusion matrix

[[175   0   0   0   0   3   0   0   0   0]
 [  0 179   0   0   0   0 